In [1]:
import os
import warnings
import sys

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from ml import Ml
from preprocess import Preprocess

In [4]:
ml = Ml()
preprocess = Preprocess()

In [5]:
df = pd.read_csv('../data/AdSmartABdata.csv')
df

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
...,...,...,...,...,...,...,...,...,...
8072,ffea24ec-cec1-43fb-b1d1-8f93828c2be2,exposed,2020-07-05,7,Generic Smartphone,6,Chrome Mobile,0,0
8073,ffea3210-2c3e-426f-a77d-0aa72e73b20f,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
8074,ffeaa0f1-1d72-4ba9-afb4-314b3b00a7c7,control,2020-07-04,9,Generic Smartphone,6,Chrome Mobile,0,0
8075,ffeeed62-3f7c-4a6e-8ba7-95d303d40969,exposed,2020-07-05,15,Samsung SM-A515F,6,Samsung Internet,0,0


In [ ]:
# Get URL from DVC
# path = 'data/AdSmartABdata.csv'
# repo = 'https://github.com/jedisam/abtest-mlops'
# version = '6db449393c9626c4fbca44946dfa103660685a27'

In [ ]:
# Load data from dvc using the dvc.api.Dataset class
# data_url = dvc.api.get_url(
#     path=path,
#     repo=repo,
#     rev=version
# )
# data_url

In [ ]:
# Read CSV file from remote repository
# data = pd.read_csv(data_url, sep=',')
# data

In [6]:
# change the date column to datetime
# from preprocess import Preprocess
data = preprocess.convert_to_datetime(df, 'date')
data

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
...,...,...,...,...,...,...,...,...,...
8072,ffea24ec-cec1-43fb-b1d1-8f93828c2be2,exposed,2020-07-05,7,Generic Smartphone,6,Chrome Mobile,0,0
8073,ffea3210-2c3e-426f-a77d-0aa72e73b20f,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
8074,ffeaa0f1-1d72-4ba9-afb4-314b3b00a7c7,control,2020-07-04,9,Generic Smartphone,6,Chrome Mobile,0,0
8075,ffeeed62-3f7c-4a6e-8ba7-95d303d40969,exposed,2020-07-05,15,Samsung SM-A515F,6,Samsung Internet,0,0


# Exploring the categorical columns

In [7]:
numerical_column = preprocess.get_numerical_columns(df)
categorical_column = preprocess.get_categorical_columns(df)

In [ ]:
top2_platform = relevant_rows['platform_os'].value_counts().index.to_list()[:2]
top2_platform

In [ ]:
platform_list = []
for platform in top2_platform:
    platforms = relevant_rows.loc[relevant_rows['platform_os'] == platform]
    platform_list.append(platforms)
platform_six = platform_list[0]
platform_five = platform_list[1]

In [ ]:
platform_six.to_csv('../data/platform_six.csv')
platform_five.to_csv('../data/platform_five.csv')

In [ ]:
top2_browser = relevant_rows['browser'].value_counts().index.to_list()[:2]
top2_browser

In [ ]:
browser_list = []
for browser in top2_browser:
    browsers = relevant_rows.loc[relevant_rows['browser'] == browser]
    browser_list.append(browsers)
chrome_mobile_data = browser_list[0]
chrome_mobile_webview_data = browser_list[1]

In [ ]:
chrome_mobile_data.to_csv('../data/chrome_mobile_data.csv')
chrome_mobile_webview_data.to_csv('../data/chrome_mobile_webview_data.csv')

In [ ]:
df2 = relevant_rows.drop('no', axis=1)
# df2

In [ ]:
df2.rename(columns = {'yes': 'clicked_or_not'}, inplace=True)
# df2

# Splitting Numerical and categorical variables

In [ ]:
numerical_column = df2.select_dtypes(exclude="object").columns.tolist()
categorical_column = df2.select_dtypes(include="object").columns.tolist()
print("Numerical Columns:", numerical_column)
print("****************")
print("Categorical Columns:", categorical_column)

# Splitting the columns for one hot encoding and label encoding

In [ ]:
to_one_hot_encoding = [col for col in categorical_column if df2[col].nunique() <= 10 and df2[col].nunique() > 2]

# Get Categorical Column names thoose are not in "to_one_hot_encoding"
to_label_encoding = [col for col in categorical_column if not col in to_one_hot_encoding]

print("To One Hot Encoding:", to_one_hot_encoding)
print("To Label Encoding:", to_label_encoding)


In [ ]:
# We will use built in pandas function "get_dummies()" to simply to encode "to_one_hot_encoding" columns
one_hot_encoded_columns = pd.get_dummies(df2[to_one_hot_encoding])
# one_hot_encoded_columns

In [ ]:
le = LabelEncoder()
df2[to_label_encoding] = df2[to_label_encoding].apply(le.fit_transform)
# df2

In [ ]:
# Label Encoding

label_encoded_columns = []
# For loop for each columns
for col in to_label_encoding:
    # We define new label encoder to each new column
    le = LabelEncoder()
    # Encode our data and create new Dataframe of it, 
    # notice that we gave column name in "columns" arguments
    column_dataframe = pd.DataFrame(le.fit_transform(df2[col]), columns=[col] )
    # and add new DataFrame to "label_encoded_columns" list
    label_encoded_columns.append(column_dataframe)

# Merge all data frames
label_encoded_columns = pd.concat(label_encoded_columns, axis=1)
# label_encoded_columns

In [ ]:
# Copy our DataFrame to X variable
X = df2.copy()
X.drop(['date', 'browser'], axis=1, inplace=True)
# Merge DataFrames
X = pd.concat([X, one_hot_encoded_columns], axis=1)
print("All columns:", X.columns.tolist())
# X

In [ ]:
# Define Y (This is the value we will predict)
y = X["yes"]

# Droping "class" from X
X.drop(["yes"], axis=1, inplace=True)
# X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.23)

In [ ]:
# print(X_train.shape)
# print(X_test.shape)
# print(X_val.shape)

# Logistic Regression

In [ ]:
# Define Logistic Regression Model
log = LogisticRegression()
# We fit our model with our train data
log.fit(X_train, y_train)
# Then predict results from X_test data
pred_log = log.predict(X_test)

# See First 10 Predictions and They Actual Values
print("Predicted:", pred_log[0:10])
print("Actual:", y_test[0:10])

In [ ]:
from matplotlib import pyplot
# get importance
importance = log.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

## Confusion Matrices

In [ ]:
cm_log = confusion_matrix(y_test, pred_log)
print("Logistic Regression:\n", cm_log)

## Accuracy scores

In [ ]:
# Logistic Regression
acc_log = accuracy_score(y_test, pred_log)
print("Logistic Regression accuracy:", acc_log)